In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

LANGCHAIN_TRACING_V2 = os.getenv("LANGCHAIN_TRACING_V2")
LANGCHAIN_ENDPOINT = os.getenv("LANGCHAIN_ENDPOINT")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

In [4]:
import os
from langchain_openai import OpenAI

In [3]:
#from langchain.chat_models import AzureChatOpenAI
from langchain_openai import AzureChatOpenAI

gpt = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo-1106",
    openai_api_key="7c3f9550b69c419aa0f1830e338ff562",
    openai_api_type="azure",
    openai_api_version="2023-12-01-preview",
    azure_endpoint="https://chatbotopenaikeyswe.openai.azure.com/",
    verbose=True,
)

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import CSVLoader
path = "C:/Users/Admin/Desktop/hsag chatbot/ragcsvdata"
csv_loader_kwargs={'autodetect_encoding': True}
loader = DirectoryLoader(path, glob="**/*.csv", loader_cls=CSVLoader, loader_kwargs=csv_loader_kwargs)
db = loader.load()

In [6]:
from langchain_community.document_loaders import AsyncHtmlLoader

urls = ["https://www.thuega-energie-gmbh.de/privatkunden.html"]
loader = AsyncHtmlLoader(urls)
docs = loader.load()

from langchain_community.document_transformers import Html2TextTransformer

html2text = Html2TextTransformer()
db += html2text.transform_documents(docs)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  3.08it/s]


In [7]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
from langchain_openai import AzureOpenAIEmbeddings
embed = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002",
    openai_api_key="7c3f9550b69c419aa0f1830e338ff562",
    azure_endpoint="https://chatbotopenaikeyswe.openai.azure.com/"
)

In [9]:
#vectorstore = Chroma.from_documents(texts, embed, persist_directory="C:/chroma_db")
#vectorstore.persist()

vectorstore2 = Chroma(persist_directory="C:/chroma_db", embedding_function=embed)
retriever = vectorstore2.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [11]:
from langchain.prompts import ChatPromptTemplate

template = """
You are an assistant named "hsag-chatbot" for question-answering tasks 
related to the Energy industry and general conversation. 

If the question is related to the energy domain try to answer the question from the knowledge 
you have in your memory.

Use the following pieces of retrieved context to answer the question 
or engage in small talk with the user in a friendly and informative way. 
If you don't know the answer to a factual question, 
just say that you don't know. 
Use three sentences maximum and keep the answer concise. 
If the user asks funny questions or jokes, try to answer them 
using your knowledge or generate a humorous response. 
If they ask general knowledge about the world, try to answer 
those questions using your knowledge and also domain-specific knowledge 
about the energy industry. 


Context:
{context}

Question:
{question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [12]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm_model = gpt

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": prompt | llm_model, "context": itemgetter("context")}
)

In [82]:
question = "What are PV modules "

result = rag_chain.invoke({"question" : question})

print(result["response"].content)
print(result["context"])

PV modules, also known as photovoltaic modules, are the main building blocks of a solar power system. They are made up of multiple solar cells that convert sunlight into electricity, and are often used to generate renewable energy for homes and businesses. PV modules are an essential component of solar panel installations.
[Document(page_content='Direkt zur Hauptnavigation springen  Direkt zum Inhalt springen\n\n  * Privatkunden \n    * Strom\n    * Erdgas\n    * Wärme\n    * Photovoltaik\n    * E-Mobilität\n    * Internet\n    * Energieberatung\n    * Förderprogramme\n  * Geschäftskunden \n    * Strom\n      * Gewerbekunden\n      * Industrie- und Großkunden\n      * Wohnungswirtschaft\n      * Kommunen\n      * Energie- und Wasserversorger\n    * Erdgas\n      * Gewerbekunden\n      * Industrie- und Großkunden\n      * Wohnungswirtschaft\n      * Kommunen\n      * Energie- und Wasserversorgung\n    * Contracting\n    * Photovoltaik\n    * E-Mobilität\n    * Telekommunikation\n    * D

In [136]:
def get_response(question):
    # Invoke the new chain with the updated structure
    results = rag_chain.invoke({"question": question})
    
    # Extract the response from the result
    response = result["response"].content
    
    #Extract the context from the result
    #context = result["context"]
    
    return response

In [15]:
def get_response(question):
    # Invoke the new chain with the updated structure
    result = rag_chain.invoke({"question": question})
    
    # Extract the response from the result
    response = result["response"].content
    
    # Extract the context from the result
    context = result["context"]
    
    # Return the response in the desired structure
    return {
        "answer": response,
        "contexts": [str(doc) for doc in context]
    }

In [14]:
question = "What are PV Modules? And to which regions does Thuga Energie serve?"
response = get_response(question)
print("Answer:", response["answer"])
print("Contexts:", response["contexts"])

Answer: PV modules, also known as photovoltaic modules, are solar panels that convert sunlight into electricity. Thüga Energie serves the regions of Hegau-Bodensee, Rhein-Pfalz, and Allgäu-Oberschwaben.
Contexts: ["page_content='Viele Anliegen können Sie über unsere Onlineservices selbst erledigen.  \\nDort erhalten Sie auch Informationen zu den Energiepreisbremsen.\\n\\nZum Servicebereich\\n\\nSchließen\\n\\n  * Strom Angebote\\n  * Erdgas Angebote\\n\\nLeider wird dieses Gebiet nicht beliefert\\n\\nPostleitzahl\\n\\n __1  Person(en) im Haushalt\\n\\nWählen Sie die Personenzahl, oder tragen Sie Ihren Jahresverbrauch ein. (nur\\nZahlen zulässig)\\n\\nkWh im Jahr\\n\\nAktionscode\\n\\nLeider wird dieses Gebiet nicht beliefert\\n\\nPostleitzahl\\n\\n __50 m2 Wohnfläche\\n\\nkWh im Jahr\\n\\nDieses Feld wird benötigt und darf nur Zahlen enthalten\\n\\nAktionscode\\n\\n#  Ihr Ansprechpartner rund um das Thema Energie\\n\\nWir, die Thüga Energie, sind Ihr **Energie\\xadversorgungs\\xadunter

Langsmith Evaluation


In [29]:
%pip install -U langchain langsmith

   ---------------------------------------- 0.0/973.5 kB ? eta -:--:--
   ----- ---------------------------------- 122.9/973.5 kB 3.6 MB/s eta 0:00:01
   ----------------------- ---------------- 563.2/973.5 kB 7.1 MB/s eta 0:00:01
   ---------------------------------------  972.8/973.5 kB 7.7 MB/s eta 0:00:01
   ---------------------------------------- 973.5/973.5 kB 6.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/122.8 kB ? eta -:--:--
   ---------------------------------------- 122.8/122.8 kB 7.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/308.5 kB ? eta -:--:--
   ---------------------------------------  307.2/308.5 kB 9.6 MB/s eta 0:00:01
   ---------------------------------------- 308.5/308.5 kB 6.3 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.27
    Uninstalling langsmith-0.1.27:
      Successfully uninstalled langsmith-0.1.27
  Attempting uninstall: langchain-core
    Found existing in

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.1.3 requires langchain-core<0.2.0,>=0.1.42, but you have langchain-core 0.2.1 which is incompatible.
langchain-community 0.0.38 requires langchain-core<0.2.0,>=0.1.52, but you have langchain-core 0.2.1 which is incompatible.


In [16]:
from langsmith import Client

In [17]:
client = Client()

In [18]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langsmith.evaluation import evaluate, LangChainStringEvaluator

In [20]:
inputs = [
    "What is the Gas contract number of the customer named Sebastian?",
    "What is the meter number for meterId 1244 and what is its meter type?",
    "What is the Gas contract number, IBAN, street, start of the contract, city,of the customer named Sebastian?",
    "What is the purpose of the charging instruction in the EMobilitySkill and its connection to the charging payment aspect?",
    "What services does Thuga Energie provide as a regional partner in the Hegau-Bodensee, Rhein-Pfalz, and Allgau-Oberschwaben regions?",
    "What are the details of the meter with meterId 1244 in the Power section?",
    "What options are there for making changes in our Kundenportal, and how can open questions be resolved with Kundenservice?",
    "How do I delete historical meter readings and what metering systems automate reading collection and consumption data?",
    "What are PV Modules? And to which regions does Thuga Energie serve?",
    "Where can I find the meter number for my electricity or gas meter?"
    
]

outputs = [
    "The gas contract number for the customer named Sebastian is ES00000062980",
    "The meter number for meterId 1244 is ZEI2332DE2 and its meter type is ZTZ",
    "The Gas contract number is ES00000062980, the IBAN is DE89370400440532013000, the street is Schmiedeweg, the start of the contract is 2022-03-14, and the city is Lauenburg",
    "The purpose of the charging instruction in the EMobilitySkill is to facilitate the process of charging electric vehicles. Its connection to the charging payment aspect is to ensure that the costs of charging are properly managed and paid for.",
    "Thuga Energie provides innovative products such as Erdgas, Strom, WÃƒÂ¤rme, Photovoltaik, and MobilitÃƒÂ¤t. They also offer comprehensive Beratung and Dienstleistungen to their customers.",
    "The details of the meter with meterId 1244 in the Power section are as follows: meterNumber: ZEI2332DE2, validTo: 2023-05-04 23:59:59, readable: False, meterType: ZTZ, digitsBeforeDecimal: 5, digitsAfterDecimal: 0, registerId: 2, obis: 1-1:1.8.2",
    "The Kundenportal offers the option to make changes online at any time, and open questions can be resolved with the Kundenservice through online communication. Additionally, human colleagues are available to assist with any queries. Rechnungen are received via email, and payments can be made through direct debit or bank transfer.",
    "To delete historical meter readings, it is best to directly contact your metering point operator. Automated reading collection and consumption data are typically associated with smart metering systems, which transmit data to the smart meter gateway every 15 minutes. The data is then processed and transmitted to the respective metering point operator via the mobile network. For more detailed information, it is advisable to contact your network operator or metering point operator, whose contact details can be found on your most recent electricity or gas bill, as well as on your meter.",
    "PV modules are also known as photovoltaic modules, they are solar panels that convert sunlight into electricity. ThÃ¼ga Energie serves the regions of Hegau-Bodensee, Rhein-Pfalz, and AllgÃ¤u-Oberschwaben as part of its energy supply services.",
    "You can find the meter number for your electricity or gas meter on the front of the device, often near a barcode.",
]
qa_pairs = [{"question": q, "answer": a} for q, a in zip(inputs, outputs)]



In [90]:
client = Client()
dataset_name = "RAGEvaluation_testset1"
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="QA pairs about EnergyIndustry.",
)

In [91]:
client.create_examples(
    inputs=[{"question": q} for q in inputs],
    outputs=[{"answer": a} for a in outputs],
    dataset_id=dataset.id,
)

In [25]:
from langchain_openai import AzureChatOpenAI

gpt4 = AzureChatOpenAI(
    azure_deployment="gpt-4-128k",
    openai_api_key="7c3f9550b69c419aa0f1830e338ff562",
    openai_api_type="azure",
    openai_api_version="2023-12-01-preview",
    azure_endpoint="https://chatbotopenaikeyswe.openai.azure.com/",
    verbose=True,
)

In [22]:
# RAG chain
def predict_rag_answer(example: dict):
    """Use this for answer evaluation"""
    response = get_response(example["question"])
    return {"answer": response["answer"]}

In [23]:
def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = get_response(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

Correctness: QA evaluation

In [97]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langchain_core.prompts.prompt import PromptTemplate
eval_llm = gpt4

qa_evaluator = [LangChainStringEvaluator("qa", config={"llm": eval_llm},
                                         prepare_data=lambda run, example: {
                                             "prediction": run.outputs["answer"],
                                             "reference": example.outputs["answer"],
                                             "input": example.inputs["question"],
                                         } )]

dataset_name = "RAGEvaluation_testset1"

experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators= qa_evaluator,
    experiment_prefix="rag-qa-oai-qaevaluation",
    
    
)

View the evaluation results for experiment: 'rag-qa-oai-qaevaluation-17534cf6' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/a3d891b4-fc65-4205-a80f-6929ca34e386/compare?selectedSessions=9d6c4637-3f7a-48d7-9b9f-a29f89d232cc




10it [00:08,  1.19it/s]


COT_QA Evaluator

In [98]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langchain_core.prompts.prompt import PromptTemplate
eval_llm = gpt4


cot_qa_evaluator = [LangChainStringEvaluator("cot_qa", config={"llm": eval_llm},
                                         prepare_data=lambda run, example: {
                                             "prediction": run.outputs["answer"],
                                             "reference": example.outputs["answer"],
                                             "input": example.inputs["question"],
                                         } )]
#qa_evalulator = [LangChainStringEvaluator("cot_qa")]
dataset_name = "RAGEvaluation_testset1"

experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators= cot_qa_evaluator,
    experiment_prefix="rag-qa-oai-cotqa",
    
    
)

View the evaluation results for experiment: 'rag-qa-oai-cotqa-db11207f' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/a3d891b4-fc65-4205-a80f-6929ca34e386/compare?selectedSessions=9124ed0f-c34e-456d-b777-7d6f888e2c87




10it [00:28,  2.85s/it]


Contextual accuracy

In [105]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langchain_core.prompts.prompt import PromptTemplate
eval_llm = gpt4


context_qa_evaluator = [LangChainStringEvaluator("context_qa", config={"llm": eval_llm},
                                         prepare_data=lambda run, example: {
                                             "prediction": run.outputs["answer"],
                                             "reference": example.outputs["answer"],
                                             "input": example.inputs["question"],
                                         } )]
#qa_evalulator = [LangChainStringEvaluator("cot_qa")]
dataset_name = "RAGEvaluation_testset1"

experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators= context_qa_evaluator,
    experiment_prefix="rag-qa-oai-contextual accuracy",
    
    
)

View the evaluation results for experiment: 'rag-qa-oai-contextual accuracy-507b7bfe' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/a3d891b4-fc65-4205-a80f-6929ca34e386/compare?selectedSessions=b8746b22-1ec3-466d-b273-6f345c4cc5e7




10it [00:08,  1.16it/s]


Answer Correctness Evaluator

In [59]:
from langchain import hub
from langchain_openai import ChatOpenAI

# Grade prompt 
grade_prompt_answer_accuracy = prompt = hub.pull("langchain-ai/rag-answer-vs-reference")

def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """
    
    # Get summary
    input_question = example.inputs["question"]
    reference = example.outputs["answer"]
    prediction = run.outputs["answer"]

    # LLM grader
    llm = gpt4

    # Structured prompt
    
    answer_grader = grade_prompt_answer_accuracy | llm

    # Get score
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_score", "score": score}

In [60]:
from langsmith.evaluation import evaluate

dataset_name = "RAGEvaluation_testset1"
experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators=[answer_evaluator],
    experiment_prefix="rag-qa-oai",
)

View the evaluation results for experiment: 'rag-qa-oai-144ff955' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/51858942-f9d0-4726-98ab-3b095d3d56d3/compare?selectedSessions=7b3631db-c9d6-4a59-a3bd-5c3c0fe2b816




0it [00:00, ?it/s]c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The method `ChatOpenAI.with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
10it [00:19,  1.90s/it]


Hallucination

In [100]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate
eval_llm = gpt4

answer_hallucination_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "llm": eval_llm,
        "criteria": {
            "accuracy": """Is the Assistant's Answer grounded in the Ground Truth documentation? 
            A score of 0 means that the Assistant answer contains is not at all based upon / grounded in the
            Ground Truth documentation. A score of 5 means that the Assistant score contains some information
            (e.g., a hallucination) that is not captured in the  Ground Truth documentation. A score of 10 means 
            that the Assistant answer is fully based upon / grounded in the Ground Truth documentation."""
        },
        "normalize_by": 10, 
        
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["contexts"],
        "input": example.inputs["question"],
    }
)
    


In [102]:
dataset_name = "RAGEvaluation_testset1"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_hallucination_evaluator],
    experiment_prefix="rag-qa-oai-hallucination",
)

View the evaluation results for experiment: 'rag-qa-oai-hallucination-9094677e' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/a3d891b4-fc65-4205-a80f-6929ca34e386/compare?selectedSessions=431e3641-0a93-410b-a0e4-c496e07cf70f




2it [00:11,  5.06s/it]Error running evaluator <DynamicRunEvaluator evaluate> on run 6899ca98-af22-42dc-a825-b5919f28a4cc: ValueError("Invalid output: The Assistant's answer is not based on the Ground Truth information provided. The Ground Truth documentation contains specific log data and metadata related to a conversation ID and a meter reading record, but it does not provide instructions or information on how to delete historical meter readings or details about metering systems that automate reading collection and consumption data. The Assistant's response appears to be a generic explanation that is not grounded in the specifics of the provided Ground Truth.\n\nGiven that the response is not supported by the Ground Truth documentation, the rating would be as follows:\n\n[[0]]. Output must contain a double bracketed string                 with the verdict between 1 and 10.")
Traceback (most recent call last):
  File "c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-pack

Document Relevance to the question

In [161]:
grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")

def docs_relevance_evaluator(run, example) -> dict:
    """
    A simple evaluator for document relevance
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    llm = gpt4

    # Structured prompt
    answer_grader = grade_prompt_doc_relevance | llm

    # Get score
    score = answer_grader.invoke({"question":input_question,
                                  "documents":contexts})
    score = score["Score"]

    return {"key": "document_relevance", "score": score}

In [103]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

eval_llm = gpt4

docs_relevance_evaluator = LangChainStringEvaluator(
    "score_string",
    config={
        "llm": eval_llm,
        "criteria": {
            "accuracy": """The Assistant's Answer is a set of documents retrieved from a vectorstore. The input is
            a question used for retrieval. You will score whether the Assistant's answer (retrieved docs) are relevant to 
            the input question. A score of [[0]] means that the Assistant answer contains documents that are not at all
            relevant to the input question. A score of[[5]] means that the Assistant answer contains some documents 
            are relevant to the input question. A score of [[10]] means that all of the Assistant answer documents are all
            relevant to the input question"""
        },
        "normalize_by": 10,
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["contexts"],
        "input": example.inputs["question"],
    }
)


In [104]:
dataset_name = "RAGEvaluation_testset1"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[docs_relevance_evaluator],
    experiment_prefix="rag-qa-oai-doc-relevance",
)

View the evaluation results for experiment: 'rag-qa-oai-doc-relevance-8bd4b0bc' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/a3d891b4-fc65-4205-a80f-6929ca34e386/compare?selectedSessions=9b0e1559-c592-49b0-ad5d-8ff896241b5c




5it [00:10,  1.27s/it]Error running evaluator <DynamicRunEvaluator evaluate> on run 2797623d-e3dc-4673-ae3c-3c2b844f2746: ValueError("Invalid output: The Assistant's answer appears to be a set of excerpts from some form of dataset logs and CSV data related to meter readings. The first two excerpts are from a system log with metadata about the conversation, likely related to meter reading queries but do not provide any information on how to delete historical meter readings or what systems automate reading collection. The third excerpt provides details of a specific meter reading from a CSV dataset, including meterId, meterNumber, and other technical details.\n\nNone of the provided excerpts directly answer the user's question about deleting historical meter readings or mention any metering systems that automate reading collection and consumption data. The response seems to be system-generated logs and data which are not intended to be informative answers to the question posed. Therefore

In [21]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

eval_llm = gpt4

# Define the evaluation criteria for various aspects of the RAG model's output
criteria = {
    "accuracy": """Is the Assistant's Answer grounded in the Ground Truth documentation?
    A score of 0 means that the Assistant's answer is not at all based upon/grounded in the
    Ground Truth documentation. A score of 10 means that the Assistant's answer is fully based upon/grounded in the Ground Truth documentation.""",
    
    "completeness": """Does the Assistant's Answer cover all aspects of the question asked?
    A score of 0 means that the Assistant's answer misses most of the key points required to fully address the question.
    A score of 10 means that the Assistant's answer thoroughly covers all aspects of the question.""",
    
    "coherence": """Is the Assistant's Answer logically coherent and easy to understand?
    A score of 0 means that the Assistant's answer is disjointed and difficult to follow.
    A score of 10 means that the Assistant's answer is logically structured and easy to follow."""
}

# Create the evaluator
rag_model_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "llm": eval_llm,
        "criteria": criteria,
        "normalize_by": 10,
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["contexts"],
        "input": example.inputs["question"],
    }
)

# Define the dataset name
dataset_name = "RAGEvaluation_testset1"

# Evaluate the RAG model using the defined evaluators
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[rag_model_evaluator],
    experiment_prefix="rag-qa-oai-full-evaluation",
)


C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'rag-qa-oai-full-evaluation-7bdf657c' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/a3d891b4-fc65-4205-a80f-6929ca34e386/compare?selectedSessions=0c2c446b-d034-42ce-84c8-9ad7fe517cb3




6it [00:27,  3.76s/it]Error running evaluator <DynamicRunEvaluator evaluate> on run 60b1b64c-e485-4e43-9a35-befedf3716fd: ValueError("Invalid output: The Assistant's answer does not refer directly to the Ground Truth documentation provided, which does not include information on how to delete historical meter readings or details about metering systems that automate reading collection and consumption data. The response provided by the Assistant instead offers a general explanation of the process one might follow to delete historical meter readings and describes common technologies used in automated metering systems. While this might be useful information in a broader context, it does not adhere to the Ground Truth documents.\n\nAccuracy: The answer is not grounded in the provided Ground Truth documentation, as the documents do not contain information related to the deletion of historical meter readings or specific metering systems that automate the collection of readings and consumption 

In [30]:
# Step 3: Define Prompt Template
template = """
You are an assistant named "hsag-chatbot" for question-answering tasks 
related to the Energy industry and general conversation. 

If the question is related to the energy domain try to answer the question from the knowledge 
you have in your memory.

Use the following pieces of retrieved context to answer the question 
or engage in small talk with the user in a friendly and informative way. 
If you don't know the answer to a factual question, 
just say that you don't know. 
Use three sentences maximum and keep the answer concise. 
If the user asks funny questions or jokes, try to answer them 
using your knowledge or generate a humorous response. 
If they ask general knowledge about the world, try to answer 
those questions using your knowledge and also domain-specific knowledge 
about the energy industry. 

Context:
{context}

Question:
{question}
"""

# Use the ChatPromptTemplate to create the prompt
prompt = ChatPromptTemplate.from_template(template)

# Step 4: Define Evaluation Criteria
criteria = {
    "clarity": """How clear and understandable is the prompt?
    A score of 0 means the prompt is very unclear and confusing. A score of 10 means the prompt is very clear and easy to understand.""",
    
    "specificity": """How specific is the prompt in guiding the model towards the desired response?
    A score of 0 means the prompt is very vague. A score of 10 means the prompt is very specific.""",
    
    "brevity": """Is the prompt concise and to the point without unnecessary information?
    A score of 0 means the prompt is verbose. A score of 10 means the prompt is very concise.""",
    
    "relevance": """How relevant is the prompt to the task at hand?
    A score of 0 means the prompt is not relevant at all. A score of 10 means the prompt is highly relevant.""",
    
    "effectiveness": """How effectively does the prompt elicit the desired response from the model?
    A score of 0 means the prompt does not effectively guide the model. A score of 10 means the prompt very effectively guides the model."""
}

prompt_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "llm": eval_llm,
        "criteria": criteria,
        "normalize_by": 10,
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["answer"],
        "reference": run.outputs["contexts"],
        "input": example.inputs["question"],
    }
)
dataset_name = "RAGEvaluation_testset1"


experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[prompt_evaluator],
    experiment_prefix="prompt-evaluation-oo1",
)


View the evaluation results for experiment: 'prompt-evaluation-eb641060' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/a3d891b4-fc65-4205-a80f-6929ca34e386/compare?selectedSessions=155b330b-7eef-4594-becb-356fcb61fb0c




10it [00:32,  3.24s/it]


In [33]:
# Step 3: Define Prompt Template
template = """
You are an assistant named "hsag-chatbot" for question-answering tasks 
related to the Energy industry and general conversation. 

If the question is related to the energy domain try to answer the question from the knowledge 
you have in your memory.

Use the following pieces of retrieved context to answer the question 
or engage in small talk with the user in a friendly and informative way. 
If you don't know the answer to a factual question, 
just say that you don't know. 
Use three sentences maximum and keep the answer concise. 
If the user asks funny questions or jokes, try to answer them 
using your knowledge or generate a humorous response. 
If they ask general knowledge about the world, try to answer 
those questions using your knowledge and also domain-specific knowledge 
about the energy industry. 

Context:
{context}

Question:
{question}
"""

# Use the ChatPromptTemplate to create the prompt
prompt = ChatPromptTemplate.from_template(template)

# Step 4: Define Evaluation Criteria
criteria = {
    "clarity": """How clear and understandable is the prompt?
    A score of 0 means the prompt is very unclear and confusing. A score of 10 means the prompt is very clear and easy to understand.""",
    
    "specificity": """How specific is the prompt in guiding the model towards the desired response?
    A score of 0 means the prompt is very vague. A score of 10 means the prompt is very specific.""",
    
    "brevity": """Is the prompt concise and to the point without unnecessary information?
    A score of 0 means the prompt is verbose. A score of 10 means the prompt is very concise.""",
    
    "relevance": """How relevant is the prompt to the task at hand?
    A score of 0 means the prompt is not relevant at all. A score of 10 means the prompt is highly relevant.""",
    
    "effectiveness": """How effectively does the prompt elicit the desired response from the model?
    A score of 0 means the prompt does not effectively guide the model. A score of 10 means the prompt very effectively guides the model."""
}

prompt_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "llm": eval_llm,
        "criteria": criteria,
        "normalize_by": 10,
    },
    prepare_data=lambda run, example: {
        "prediction": prompt.format(context="", question=run.outputs["answer"]),
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)
dataset_name = "RAGEvaluation_testset1"


experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[prompt_evaluator],
    experiment_prefix="prompt-evaluation-oo1",
)


View the evaluation results for experiment: 'prompt-evaluation-oo1-ad221f99' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/a3d891b4-fc65-4205-a80f-6929ca34e386/compare?selectedSessions=918b6323-0728-4055-ba97-9c773fe61e31




10it [00:35,  3.60s/it]


In [41]:
# Step 3: Define Prompt Template
template = """
You are an assistant named "hsag-chatbot" for question-answering tasks 
related to the Energy industry and general conversation. 

If the question is related to the energy domain try to answer the question from the knowledge 
you have in your memory.

Use the following pieces of retrieved context to answer the question 
or engage in small talk with the user in a friendly and informative way. 
If you don't know the answer to a factual question, 
just say that you don't know. 
Use three sentences maximum and keep the answer concise. 
If the user asks funny questions or jokes, try to answer them 
using your knowledge or generate a humorous response. 
If they ask general knowledge about the world, try to answer 
those questions using your knowledge and also domain-specific knowledge 
about the energy industry. 

Context:
{context}

Question:
{question}
"""

# Use the ChatPromptTemplate to create the prompt
prompt = ChatPromptTemplate.from_template(template)

# Step 4: Define Evaluation Criteria
criteria = {
    "clarity": """How clear and understandable is the prompt template?
    A score of 0 means the prompt is very unclear and confusing. A score of 10 means the prompt is very clear and easy to understand.""",
    
    "specificity": """How specific is the prompt template in guiding the model towards the desired response?
    A score of 0 means the prompt is very vague. A score of 10 means the prompt is very specific.""",
    
    "brevity": """Is the prompt template concise and to the point without unnecessary information?
    A score of 0 means the prompt is verbose. A score of 10 means the prompt is very concise.""",
    
    "relevance": """How relevant is the prompt template to the task at hand?
    A score of 0 means the prompt is not relevant at all. A score of 10 means the prompt is highly relevant.""",
    
    "effectiveness": """How effectively does the prompt template elicit the desired response from the model?
    A score of 0 means the prompt does not effectively guide the model. A score of 10 means the prompt very effectively guides the model."""
}

prompt_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "llm": eval_llm,
        "criteria": criteria,
        "normalize_by": 10,
    },
    prepare_data=lambda run, example: {
        "prediction": prompt.format(
            context=run.outputs.get("contexts", ""),  # Use contexts if available, otherwise set to empty string
            question=run.outputs["answer"]
        ),
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

   




dataset_name = "RAGEvaluation_testset1"


experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[prompt_evaluator],
    experiment_prefix="prompt-evaluation-oo1",
)


View the evaluation results for experiment: 'prompt-evaluation-oo1-c249960d' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/a3d891b4-fc65-4205-a80f-6929ca34e386/compare?selectedSessions=7d3a4392-43c8-4775-8a87-780f052d658c




3it [00:19,  4.85s/it]Error running evaluator <DynamicRunEvaluator evaluate> on run a24b9b91-5c17-418f-8eaa-9c37c8092425: ValueError('Invalid output: The provided response is not an actual answer to the user\'s question but rather an instruction to the AI on how to behave as an assistant. The user asks about options for making changes in the Kundenportal and resolving open questions with Kundenservice, while the response gives a directive to the AI named "hsag-chatbot" on how to answer questions.\n\nExplanation:\n- Clarity: The prompt template provided to the AI is clear in its instructions on what the AI should do when answering questions. However, it does not answer the user\'s question, so it cannot be rated highly in clarity for the user\'s perspective.\n- Specificity: The prompt is specific in guiding the AI on its general behavior when interacting with users, but it does not provide specific guidance for the user\'s question, making it off-target.\n- Brevity: The prompt is not br

In [27]:
# Step 3: Define Prompt Template
template = """
You are an assistant named "hsag-chatbot" for question-answering tasks 
related to the Energy industry and general conversation. 

If the question is related to the energy domain try to answer the question from the knowledge 
you have in your memory.

Use the following pieces of retrieved context to answer the question 
or engage in small talk with the user in a friendly and informative way. 
If you don't know the answer to a factual question, 
just say that you don't know. 
Use three sentences maximum and keep the answer concise. 
If the user asks funny questions or jokes, try to answer them 
using your knowledge or generate a humorous response. 
If they ask general knowledge about the world, try to answer 
those questions using your knowledge and also domain-specific knowledge 
about the energy industry. 

Context:
{context}

Question:
{question}

Answer:
{answer}
"""
eval_llm = gpt4
# Use the ChatPromptTemplate to create the prompt
prompt = ChatPromptTemplate.from_template(template)

# Step 4: Define Evaluation Criteria
criteria = {
    "clarity": """How clear and understandable is the prompt template?
    A score of 0 means the prompt is very unclear and confusing. A score of 10 means the prompt is very clear and easy to understand.""",
    
    "specificity": """How specific is the prompt template in guiding the model towards the desired response?
    A score of 0 means the prompt is very vague. A score of 10 means the prompt is very specific.""",
    
    "brevity": """Is the prompt template concise and to the point without unnecessary information?
    A score of 0 means the prompt is verbose. A score of 10 means the prompt is very concise.""",
    
    "relevance": """How relevant is the prompt template to the task at hand?
    A score of 0 means the prompt is not relevant at all. A score of 10 means the prompt is highly relevant.""",
    
    "effectiveness": """How effectively does the prompt template elicit the desired response from the model?
    A score of 0 means the prompt does not effectively guide the model. A score of 10 means the prompt very effectively guides the model."""
}

prompt_evaluator = LangChainStringEvaluator(
    "labeled_score_string",
    config={
        "llm": eval_llm,
        "criteria": criteria,
        "normalize_by": 10,
    },
    prepare_data=lambda run, example: {
        "prediction": prompt.format(
            context=run.outputs.get("contexts", ""),  # Use contexts if available, otherwise set to empty string
            answer=run.outputs["answer"],
            question=example.inputs["question"]
        ),
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

   




dataset_name = "RAGEvaluation_testset1"


experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[prompt_evaluator],
    experiment_prefix="prompt-evaluation-oo1",
)


View the evaluation results for experiment: 'prompt-evaluation-oo1-ba41f292' at:
https://smith.langchain.com/o/193e2b09-24bf-506a-879c-4ee7e4127ca4/datasets/a3d891b4-fc65-4205-a80f-6929ca34e386/compare?selectedSessions=cd06b9c6-6e7d-4ba0-bec5-b9680b1384dd




10it [00:52,  5.22s/it]
